In [16]:
import pandas as pd
import numpy as np

In [17]:
PATH = "/home/STual/KG-cadastre/data/SKOS_ATTRIBUTES.csv"

skos_attr = pd.read_csv(PATH,header=0)
display(skos_attr)

,skos:Concept,rdf:type,skos:inScheme,skos:prefLabel@fr,skos:altLabel@fr,skos:prefLabel@en,skos:altLabel@en,skos:definition@fr,skos:broader
0,PlotAttribute,add:AttributeType,CadastreAttributeList,Attribut de parcelle,NaN,Plot attribute,NaN,Attribut de parcelle,NaN
1,PlotOwner,add:AttributeType,CadastreAttributeList,Propriétaire,Propriétaire ou usufruitier,Plot owner,NaN,Propriétaire(s) ou usufruitier(s) d'une parcel...,PlotAttribute
2,PlotNature,add:AttributeType,CadastreAttributeList,Nature,NaN,Plot nature,NaN,Nature d'une parcelle,PlotAttribute
3,Mention,add:AttributeType,CadastreAttributeList,Mention,NaN,Mention,NaN,Mention d'une personne/d'une parcelle dans un ...,NaN
4,OwnerAttribute,add:AttributeType,CadastreAttributeList,Attribut de personne,NaN,Owner attribute,NaN,Attribut de personne (propriétaire ou usufruit...,NaN
5,OwnerIdentity,add:AttributeType,CadastreAttributeList,Identité complète,NaN,Full identity,NaN,"Identité complète (nom-prénoms attachés, raiso...",OwnerAttribute
6,OwnerFirstName,add:AttributeType,CadastreAttributeList,Prénom,NaN,First name,NaN,Prénom(s) d'une personne,OwnerAttribute
7,OwnerLastName,add:AttributeType,CadastreAttributeList,Nom de famille,NaN,Last name,NaN,Nom de famille d'une personne,OwnerAttribute
8,OwnerOtherLabel,add:AttributeType,CadastreAttributeList,Autre désignation,Expression référentielle,Other label,NaN,"Expression référentielle, surnom, titre de nob...",OwnerAttribute
9,OwnerTitle,add:AttributeType,CadastreAttributeList,Titre de civilité,Civilité,Title,NaN,"Civilité (Monsieur, Madame, Mademoiselle)",OwnerAttribute


In [18]:
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import SKOS, RDF, OWL

# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/codes/cadastre/attributeType/")
g.bind('cad_atype', baseuri)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
add = Namespace("http://rdf.geohistoricaldata.org/def/address#")
g.bind('cad', cad)
g.bind('add', add)

# Iterate over each row in the DataFrame
for index, row in skos_attr.iterrows():
    subject_uri = URIRef(baseuri + f"{str(row['skos:Concept'])}")
    g.add((subject_uri, RDF.type, OWL.NamedIndividual))
    g.add((subject_uri, RDF.type, add.AttributeType))
    g.add((subject_uri, RDF.type, SKOS.Concept))
    g.add((subject_uri, SKOS.inScheme, URIRef(cad + f"{str(row['skos:inScheme'])}")))
    g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@fr"],lang='fr')))

    if row["skos:altLabel@fr"] == row["skos:altLabel@fr"]:
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@fr"],lang='fr')))

    if row["skos:prefLabel@en"] == row["skos:prefLabel@en"]:
        g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@en"],lang='en')))

    if row["skos:altLabel@en"] == row["skos:altLabel@en"]:
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@en"],lang='en')))

    if row["skos:broader"] == row["skos:broader"]:
        g.add((subject_uri, SKOS.broader, URIRef(baseuri + f"{str(row['skos:broader'])}")))

    if row["skos:definition@fr"] == row["skos:definition@fr"]:
        g.add((subject_uri, SKOS.definition, Literal(row["skos:definition@fr"],lang='fr')))

# Print out the graph in Turtle syntax
print(g.serialize(format='turtle'))

@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

cad_atype:Mention a add:AttributeType,
        owl:NamedIndividual,
        skos:Concept ;
    skos:definition "Mention d'une personne/d'une parcelle dans un document"@fr ;
    skos:inScheme cad:CadastreAttributeList ;
    skos:prefLabel "Mention"@en,
        "Mention"@fr .

cad_atype:OwnerActivity a add:AttributeType,
        owl:NamedIndividual,
        skos:Concept ;
    skos:altLabel "Profession"@fr ;
    skos:broader cad_atype:OwnerAttribute ;
    skos:definition "Profession ou activité d'une personne"@fr ;
    skos:inScheme cad:CadastreAttributeList ;
    skos:prefLabel "Activity"@en,
        "Activité"@fr .

cad_atype:OwnerAddress a add:AttributeType,
        owl:NamedIndividual,
        skos